# Adding Surface Reflectivity

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import astropy.units as u

#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show

Loading BokehJS ...

### Connect to Opacity Database

In [2]:
opacity = jdi.opannection(wave_range=[0.3,1])

### Load blank slate

In [3]:
sum_planet = jdi.inputs()

### Set Planet & Star Properties 

In [4]:
#phase angle 
sum_planet.phase_angle(0) #radians

#define gravity
sum_planet.gravity(gravity=3.7, gravity_unit=u.Unit('m/(s**2)')) #any astropy units available


#define star 
sum_planet.star(opacity,5000,0,4.0) #pysynphot database, temp, metallicity, logg 

## Add Surface Reflectivity

In [5]:
sum_planet.atmosphere(df=pd.DataFrame({'pressure':np.logspace(-6,2,40),
                         'temperature':np.linspace(270,100,40),
                         'H2O':np.zeros(40)+0.01,
                         'CO2':np.zeros(40)+1-0.01}))

sum_planet.surface_reflect(0.3,opacity.wno)

#can also input a wavelength dependent quantity here
#fake_surface= np.linspace(0.01,0.9, opacity.nwno)
#sum_planet.surface_reflect(fake_surface,opacity.wno)

Turning off Raman for Non-H2 atmosphere


In [6]:
sum_planet.atmosphere(df=pd.DataFrame({'pressure':np.logspace(-6,2,40),
                         'temperature':np.linspace(100,270,40),
                         'H2O':np.zeros(40)+0.01,
                         'CO2':np.zeros(40)+1-0.01}))

## Create 1D Albedo Spectrum

In [7]:
df = sum_planet.spectrum(opacity)

In [8]:
wno, alb, fpfs = df['wavenumber'],df['albedo'],df['fpfs_reflected']
wno, alb = jpi.mean_regrid(wno, alb, R=150)

In [10]:
show(jpi.spectrum([wno], [alb], plot_width=500))

What about contrast units? 

In [9]:
fpfs

['Semi-major axis not supplied. If you want fpfs, add it to `star` function. ',
 'Planet Radius not supplied. If you want fpfs, add it to `gravity` function with Mass.']

## Get Contrast Units  

In order to get contrast units we have to make sure to give the `gravity` and `star` functions semi major axis, mass and radius.

In [11]:
sum_planet.star(opacity,5000,0,4.0,semi_major=1, semi_major_unit=u.Unit('au')) 
sum_planet.gravity(radius=1./3.,radius_unit=u.Unit('R_earth'), 
             mass=1./3.,mass_unit=u.Unit('M_earth')) 
df = sum_planet.spectrum(opacity)
wno, alb, fpfs = df['wavenumber'],df['albedo'],df['fpfs_reflected']
wno, fpfs = jdi.mean_regrid(wno, fpfs , R=150)


In [13]:
show(jpi.spectrum([wno], [fpfs*1e6], plot_width=500))